In [3]:
import pandas as pd
from pytrends.request import TrendReq
#import requests
#import os
#import json

# Google trends

This is done by using the *pytrends* package. This simply scrapes the google trends website.

Documentation is found here:

https://github.com/GeneralMills/pytrends

In [31]:
def get_trends(keyword, time=0):
    connection = TrendReq(hl='en-US', tz=360)
    connection.build_payload(keyword, cat=0, timeframe=time, geo='US', gprop='')
    # which data?
    # DMA = metropolitan, REGION = state
    result = connection.interest_by_region(resolution='REGION', inc_low_vol=True, inc_geo_code=False)
    return result

In [128]:
# try to handle exception for:
# ReadTimeout: HTTPSConnectionPool(host='trends.google.com', port=443): Read timed out. (read timeout=5)
def get_time(keyword):
    # startdate = "1/1/2022"
    startdate = "15/4/2022"
    range = pd.date_range(start=startdate,end=pd.to_datetime("today"))
    timeframe = range.astype("str") + " " + range.shift(periods=1).astype("str")
    # this is faster than filling up empty dataframes
    # sources (1): https://gist.github.com/Coldsp33d/ea080f580ab3a3b75c2f40c201d50164
    # sources (2): https://pandas.pydata.org/pandas-docs/version/0.21/generated/pandas.DataFrame.append.html
    data = []
    for i in timeframe:
        result = get_trends([keyword], i)[keyword].values.tolist()
        data.append(result)
        print(f"Appended {i}")
    dataframe = pd.DataFrame(data).T
    states = get_trends([keyword], "all")[keyword].index
    dataframe.set_index(states, inplace=True)
    dataframe.columns = range
    return dataframe

In [139]:
# Keywords are "subjects" constructed by the Google team
# These consolidate the results of different search terms
# COVID
# this returns results about searches such as "covid", "covid19", "covid variants", etc
print("Getting the COVID dataframe")
covid_df = get_time("Coronavirus disease 2019")
# VACCINES
# this returns results about searches such as "covid vaccine", "booster shot near me", "Pfizer-BioNTech COVID-19 vaccine"
print("\nGetting the Vaccines dataframe")
vaccine_df = get_time("COVID-19-vaccine")

Getting the COVID dataframe
Appended 2022-04-15 2022-04-16
Appended 2022-04-16 2022-04-17
Appended 2022-04-17 2022-04-18
Appended 2022-04-18 2022-04-19
Appended 2022-04-19 2022-04-20
Appended 2022-04-20 2022-04-21
Appended 2022-04-21 2022-04-22
Appended 2022-04-22 2022-04-23
Appended 2022-04-23 2022-04-24

 Getting the Vaccines dataframe
Appended 2022-04-15 2022-04-16
Appended 2022-04-16 2022-04-17
Appended 2022-04-17 2022-04-18
Appended 2022-04-18 2022-04-19
Appended 2022-04-19 2022-04-20
Appended 2022-04-20 2022-04-21
Appended 2022-04-21 2022-04-22
Appended 2022-04-22 2022-04-23
Appended 2022-04-23 2022-04-24


In [ ]:
#covid_df
#covid_df.T.stack()
#vaccine_df
#vaccine_df.T.stack()